In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [4]:
files = sorted(glob('output/*.json'), key = lambda x: int(x.split('-')[1].replace('.json', '')))
len(files)

52908

In [5]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [6]:
import string

punct = set(string.punctuation)

def remove_punct(s):
    return ''.join([c for c in s if c not in punct])

def remove_duplicate(string, n = 3):
    splitted = string.split()
    n = [splitted[i: i + n] for i in range(0, len(splitted), n)]
    already = set()
    dedup = []
    for n_ in n:
        original_n = ' '.join(n_)
        n_ = ' '.join(n_).lower()
        n_ = remove_punct(n_)
        if n_ not in already:
            dedup.append(original_n)
            already.add(n_)
    return ' '.join(dedup)

In [7]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [8]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [9]:
import mp
import copy

minimum_score = 5

def loop(files):
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = os.path.join('output-audio', f'{f_split}-{i}.mp3')
            if not os.path.exists(audio_filename):
                continue
                
            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            len_y = len(y) / sr
            if len_y > 30:
                continue
            rounded_num = f'<|{round_to_nearest_0_02(len_y):.2f}|>'
            
            if data[i]['score_ms'] > minimum_score:
                a = np.array(data[i]['predict_ms'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        try:
                            dense = CountVectorizer(ngram_range = (3,3)).fit_transform([t_]).todense()
                            repeat = (dense > 3).sum() >= 1
                            if repeat:
                                t_ = remove_duplicate(t_)
                        except:
                            if len(t_) > 100:
                                t_ = remove_duplicate(t_)
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|ms|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
                    
            
            if data[i]['score_en'] > minimum_score:
                a = np.array(data[i]['predict_en'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        try:
                            dense = CountVectorizer(ngram_range = (3,3)).fit_transform([t_]).todense()
                            repeat = (dense > 3).sum() >= 1
                            if repeat:
                                t_ = remove_duplicate(t_)
                        except:
                            if len(t_) > 100:
                                t_ = remove_duplicate(t_)
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|en|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
    return results

In [10]:
results = loop((files[:10], 0))

100%|██████████| 10/10 [00:18<00:00,  1.90s/it]


In [11]:
len(results)

708

In [12]:
results[:10]

[{'new_text': "<|startoftranscript|><|ms|><|transcribe|><|0.00|> Collab dia tak boleh kerja<|0.88|><|0.88|> Tak boleh<|1.38|><|1.38|> Kena ambil yang business class jugalah<|2.74|><|2.74|> Business class jugak<|3.48|><|3.48|> Faham faham faham<|4.64|><|4.64|> Macam tu<|5.46|><|5.46|> So gaji berbeza<|6.86|><|6.86|> Gaji berbeza<|8.50|><|8.50|> Antara<|8.76|><|8.76|> Kelas-kelas ni<|9.76|><|9.76|> Berbeza<|10.58|><|10.58|> Few thousand jugak lah<|12.58|><|12.58|> Jauh<|12.62|><|12.62|> Jauh beza dia<|13.38|><|13.38|> Beza seribu dua ribu<|14.92|><|14.92|> Macam tu lah<|15.52|><|15.52|> Total<|16.76|><|16.76|> Kalau macam Singapore<|18.76|><|18.76|> Dia macam tu juga<|19.48|><|19.48|> Tapi dia ikut qualification<|20.66|><|20.66|> You diploma business class<|22.02|><|22.02|> Ijazah<|22.96|><|22.96|> Macam tu eh<|23.62|><|23.62|> First class something like that<|24.74|><|24.74|> Dia tengok qualification<|26.20|><|26.20|> Let's say kan<|27.26|><|27.26|> Eh itu gaji gaji<|28.14|><|28.14|> Ga

In [13]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
results = mp.multiprocessing(files, loop, cores = 100)

100%|██████████| 529/529 [3:42:29<00:00, 25.23s/it]


In [24]:
len(results)

3085595

In [25]:
results[0]

{'new_text': "<|startoftranscript|><|ms|><|transcribe|><|0.02|> Collab dia tak boleh kerja<|0.90|><|0.90|> Tak boleh<|1.40|><|1.40|> Kena ambil yang business class jugalah<|2.76|><|2.76|> Business class jugak<|3.50|><|3.50|> Faham faham faham<|4.66|><|4.66|> Macam tu<|5.48|><|5.48|> So gaji berbeza<|6.88|><|6.88|> Gaji berbeza<|8.52|><|8.52|> Antara<|8.78|><|8.78|> Kelas-kelas ni<|9.78|><|9.78|> Berbeza<|10.60|><|10.60|> Few thousand jugak lah<|12.60|><|12.60|> Jauh<|12.64|><|12.64|> Jauh beza dia<|13.40|><|13.40|> Beza seribu dua ribu<|14.94|><|14.94|> Macam tu lah<|15.54|><|15.54|> Total<|16.78|><|16.78|> Kalau macam Singapore<|18.78|><|18.78|> Dia macam tu juga<|19.50|><|19.50|> Tapi dia ikut qualification<|20.68|><|20.68|> You diploma business class<|22.04|><|22.04|> Ijazah<|22.98|><|22.98|> Macam tu eh<|23.64|><|23.64|> First class something like that<|24.76|><|24.76|> Dia tengok qualification<|26.22|><|26.22|> Let's say kan<|27.28|><|27.28|> Eh itu gaji gaji<|28.16|><|28.16|> Gaj

In [26]:
import IPython.display as ipd
ipd.Audio(results[0]['audio_filename'])

In [27]:
with open('prepared-pseudolabel.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 3085595/3085595 [00:23<00:00, 133722.14it/s]


In [28]:
!ls -lh prepared-pseudolabel.jsonl

-rw-r--r-- 1 ubuntu ubuntu 1.9G Apr 26 05:55 prepared-pseudolabel.jsonl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
from huggingface_hub import HfApi
api = HfApi()

In [30]:
api.upload_file(
    path_or_fileobj='prepared-pseudolabel.jsonl',
    path_in_repo='prepared-pseudolabel.jsonl',
    repo_id='mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3-timestamp',
    repo_type='dataset',
)

'https://huggingface.co/datasets/mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3-timestamp/blob/main/prepared-pseudolabel.jsonl'

In [31]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [32]:
tokenizer.tokenize(results[-4]['new_text'])

['<|startoftranscript|>',
 '<|en|>',
 '<|transcribe|>',
 '<|0.02|>',
 'Ġline',
 '.',
 'ĠAlright',
 '.',
 'ĠOkay',
 '.',
 'ĠDal',
 'am',
 'Ġg',
 'aji',
 'Ġyang',
 'Ġbegitu',
 'Ġting',
 'gi',
 'Ġada',
 'Ġjuga',
 'Ġdia',
 'Ġpunya',
 'Ġkes',
 'us',
 'ahan',
 'Ġdia',
 'Ġlah',
 '<|7.86|>',
 '<|7.86|>',
 'Ġkan',
 '.',
 'ĠSo',
 'ĠSher',
 'ry',
 'ĠI',
 'brahim',
 'Ġlah',
 'Ġcer',
 'ita',
 'Ġkat',
 'Ġsaya',
 'Ġdia',
 'Ġpun',
 'Ġex',
 'ĠQatar',
 '.',
 'ĠDia',
 'Ġk',
 'ata',
 'Ġada',
 'Ġke',
 'f',
 'ew',
 '.',
 'ĠBet',
 'ul',
 '<|14.32|>',
 '<|14.32|>',
 'Ġtak',
 '?',
 'ĠKe',
 'f',
 'ew',
 'Ġmak',
 'n',
 'anya',
 'Ġorang',
 'Ġk',
 'ata',
 'Ġyou',
 'Ġtak',
 'Ġboleh',
 'Ġle',
 'wat',
 'Ġbal',
 'ik',
 'Ġrumah',
 '.',
 'ĠYa',
 'Ġbet',
 'ul',
 '.',
 'ĠSo',
 'Ġkalau',
 'Ġada',
 '<|19.90|>',
 '<|19.90|>',
 'Ġke',
 'f',
 'ew',
 'Ġmacam',
 'Ġyou',
 'Ġada',
 'Ġke',
 'f',
 'ew',
 'Ġjuga',
 'Ġzaman',
 '?',
 'ĠAda',
 '.',
 'ĠMas',
 'ih',
 'Ġada',
 '?',
 'ĠS',
 'amp',
 'ai',
 'Ġsekarang',
 '.',
 'ĠSi',
 'apa',
